# Following actions should be performed:
* If for any column(s), the variance is equal to zero, then you need to remove those variable(s).
* Check for null and unique values for test and train sets
* Apply label encoder.
* Perform dimensionality reduction.
* Predict your test_df values using xgboost

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
train_data=pd.read_csv('train.csv')

In [4]:
train_data.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


In [5]:
train_data.shape

(4209, 378)

In [6]:
train_data.duplicated().sum()

0

In [7]:
pd.options.display.float_format='{:,.4f}'.format
variance=train_data.var(numeric_only=True)
variance
variance=variance.reset_index()
variance.columns=['feature','values']
variance

,feature,values
0,ID,"5,941,936.1180"
1,y,160.7667
2,X10,0.0131
3,X11,0.0000
4,X12,0.0695
...,...,...
365,X380,0.0080
366,X382,0.0075
367,X383,0.0017
368,X384,0.0005


In [8]:
var=variance.loc[variance['values']==0,'feature']
train_data1=train_data.drop(var,axis=1)
train_data1.shape

(4209, 366)

In [9]:
train_data1.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.8100,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.5300,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.2600,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.6200,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.0200,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


In [10]:
train_data1.drop('ID',axis=1,inplace=True)
train_data1.shape

(4209, 365)

In [11]:
train_data1[train_data1.isnull().any(axis=1)]

,y,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385


In [12]:
train_data1

,y,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,130.8100,k,v,at,a,d,u,j,o,0,...,0,0,1,0,0,0,0,0,0,0
1,88.5300,k,t,av,e,d,y,l,o,0,...,1,0,0,0,0,0,0,0,0,0
2,76.2600,az,w,n,c,d,x,j,x,0,...,0,0,0,0,0,0,1,0,0,0
3,80.6200,az,t,n,f,d,x,l,e,0,...,0,0,0,0,0,0,0,0,0,0
4,78.0200,az,v,n,f,d,h,d,n,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,107.3900,ak,s,as,c,d,aa,d,q,0,...,1,0,0,0,0,0,0,0,0,0
4205,108.7700,j,o,t,d,d,aa,h,h,0,...,0,1,0,0,0,0,0,0,0,0
4206,109.2200,ak,v,r,a,d,aa,g,e,0,...,0,0,1,0,0,0,0,0,0,0
4207,87.4800,al,r,e,f,d,aa,l,u,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
test_data=pd.read_csv('test.csv')

In [14]:
test_data.head()

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,az,v,n,f,d,t,a,w,0,...,0,0,0,1,0,0,0,0,0,0
1,2,t,b,ai,a,d,b,g,y,0,...,0,0,1,0,0,0,0,0,0,0
2,3,az,v,as,f,d,a,j,j,0,...,0,0,0,1,0,0,0,0,0,0
3,4,az,l,n,f,d,z,l,n,0,...,0,0,0,1,0,0,0,0,0,0
4,5,w,s,as,c,d,y,i,m,0,...,1,0,0,0,0,0,0,0,0,0


In [15]:
test_data[test_data.isnull().any(axis=1)]

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385


In [16]:
cat=train_data1.select_dtypes(exclude='number')
num=train_data1.select_dtypes(include='number')

In [17]:
cat.columns

Index(['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8'], dtype='object')

In [18]:
cat['X0'].unique()

array(['k', 'az', 't', 'al', 'o', 'w', 'j', 'h', 's', 'n', 'ay', 'f', 'x',
       'y', 'aj', 'ak', 'am', 'z', 'q', 'at', 'ap', 'v', 'af', 'a', 'e',
       'ai', 'd', 'aq', 'c', 'aa', 'ba', 'as', 'i', 'r', 'b', 'ax', 'bc',
       'u', 'ad', 'au', 'm', 'l', 'aw', 'ao', 'ac', 'g', 'ab'],
      dtype=object)

In [19]:
cat['X1'].unique()

array(['v', 't', 'w', 'b', 'r', 'l', 's', 'aa', 'c', 'a', 'e', 'h', 'z',
       'j', 'o', 'u', 'p', 'n', 'i', 'y', 'd', 'f', 'm', 'k', 'g', 'q',
       'ab'], dtype=object)

In [20]:
cat['X2'].unique()

array(['at', 'av', 'n', 'e', 'as', 'aq', 'r', 'ai', 'ak', 'm', 'a', 'k',
       'ae', 's', 'f', 'd', 'ag', 'ay', 'ac', 'ap', 'g', 'i', 'aw', 'y',
       'b', 'ao', 'al', 'h', 'x', 'au', 't', 'an', 'z', 'ah', 'p', 'am',
       'j', 'q', 'af', 'l', 'aa', 'c', 'o', 'ar'], dtype=object)

In [21]:
for i in cat:
    print(cat[i].unique())
    print(i,'---'*40)

['k' 'az' 't' 'al' 'o' 'w' 'j' 'h' 's' 'n' 'ay' 'f' 'x' 'y' 'aj' 'ak' 'am'
 'z' 'q' 'at' 'ap' 'v' 'af' 'a' 'e' 'ai' 'd' 'aq' 'c' 'aa' 'ba' 'as' 'i'
 'r' 'b' 'ax' 'bc' 'u' 'ad' 'au' 'm' 'l' 'aw' 'ao' 'ac' 'g' 'ab']
X0 ------------------------------------------------------------------------------------------------------------------------
['v' 't' 'w' 'b' 'r' 'l' 's' 'aa' 'c' 'a' 'e' 'h' 'z' 'j' 'o' 'u' 'p' 'n'
 'i' 'y' 'd' 'f' 'm' 'k' 'g' 'q' 'ab']
X1 ------------------------------------------------------------------------------------------------------------------------
['at' 'av' 'n' 'e' 'as' 'aq' 'r' 'ai' 'ak' 'm' 'a' 'k' 'ae' 's' 'f' 'd'
 'ag' 'ay' 'ac' 'ap' 'g' 'i' 'aw' 'y' 'b' 'ao' 'al' 'h' 'x' 'au' 't' 'an'
 'z' 'ah' 'p' 'am' 'j' 'q' 'af' 'l' 'aa' 'c' 'o' 'ar']
X2 ------------------------------------------------------------------------------------------------------------------------
['a' 'e' 'c' 'f' 'd' 'b' 'g']
X3 -------------------------------------------------------------------

In [22]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [23]:
for col in cat.columns:
    label=LabelEncoder()
    train_data1[col]=label.fit_transform(train_data1[col])

In [24]:
train_data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4209 entries, 0 to 4208
Columns: 365 entries, y to X385
dtypes: float64(1), int32(8), int64(356)
memory usage: 11.6 MB


In [25]:
train_data_feature=train_data1.drop(columns={'y'})
train_data_target=train_data1.y
print(train_data_feature.shape)

(4209, 364)


In [26]:
#dimension reduction

In [27]:
from sklearn.decomposition import PCA
pca=PCA(n_components=.95)
pca.fit(train_data_feature,train_data_target)

PCA(n_components=0.95)

In [28]:
train_data_feature_trans=pca.fit_transform(train_data_feature)
train_data_feature_trans.shape

(4209, 6)

In [29]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error
from math import sqrt

In [30]:
train_x,test_x,train_y,test_y=train_test_split(train_data_feature_trans,train_data_target,test_size=0.30,random_state=123)
print(train_y.shape)
print(test_x.shape)
print(train_x.shape)
print(test_y.shape)

(2946,)
(1263, 6)
(2946, 6)
(1263,)


In [31]:
from xgboost import XGBRegressor
xgb_reg=XGBRegressor(objective='reg:squarederror',n_estimators=100,seed=123)
xgb_reg.fit(train_x,train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=100,
             n_jobs=None, num_parallel_tree=None, ...)

In [32]:
y_pred=xgb_reg.predict(test_x)

In [33]:
d=pd.DataFrame()
d['Actuals']=test_y
d['Predictions']=y_pred
d['Ab_Error']=abs((d['Actuals']-d['Predictions'])/d['Actuals'])
d

,Actuals,Predictions,Ab_Error
1259,105.9700,104.8700,0.0104
1793,89.2800,92.7755,0.0392
2720,98.8400,95.5077,0.0337
2024,80.2000,78.8531,0.0168
3578,98.1000,98.4592,0.0037
...,...,...,...
2362,87.1300,95.0416,0.0908
3382,91.1700,94.5959,0.0376
3404,88.2800,99.6564,0.1289
1464,96.6100,103.6033,0.0724


In [34]:
MAPE=(d['Ab_Error'].mean())*100
MAPE

7.220067681439686

MAPE < 10% Excellent Forecast

10% ≤ MAPE < 20% Good Forecast

20% ≤ MAPE < 50% Reasonable / OK Forecast

MAPE ≥ 50% Poor Forecast

# Predictions on the test data

In [37]:
test_data.columns

Index(['ID', 'X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8', 'X10',
       ...
       'X375', 'X376', 'X377', 'X378', 'X379', 'X380', 'X382', 'X383', 'X384',
       'X385'],
      dtype='object', length=377)

In [38]:
cat.columns

Index(['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8'], dtype='object')

In [39]:
pd.options.display.float_format='{:,.4f}'.format
variance1=test_data.var(numeric_only=True)
variance1
variance1=variance1.reset_index()
variance1.columns=['feature','values']
variance1

,feature,values
0,ID,"5,871,311.4840"
1,X10,0.0187
2,X11,0.0002
3,X12,0.0689
4,X13,0.0573
...,...,...
364,X380,0.0080
365,X382,0.0087
366,X383,0.0005
367,X384,0.0007


In [40]:
var1=variance1.loc[variance1['values']==0,'feature']
test_data1=test_data.drop(var1,axis=1)
print(test_data1.shape)
print(var1)


(4209, 372)
242    X257
243    X258
280    X295
281    X296
353    X369
Name: feature, dtype: object


In [41]:
X11
83      X93
97     X107
219    X233
221    X235
254    X268
275    X289
276    X290
279    X293
283    X297
315    X330
332    X347

SyntaxError: invalid syntax (1984622147.py, line 2)

In [42]:
test_data=test_data.drop(columns={'X11','X93','X107','X233','X235','X268','X289','X290','X293','X297','X330','X347'})

In [44]:
test_data.shape

(4209, 365)

In [46]:
for col in cat.columns:
    label=LabelEncoder()
    test_data[col]=label.fit_transform(test_data[col])

In [48]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4209 entries, 0 to 4208
Columns: 365 entries, ID to X385
dtypes: int32(8), int64(357)
memory usage: 11.6 MB


In [52]:
test_data.shape

(4209, 365)

In [54]:
test_data_feature=test_data.drop('ID',axis=1)
test_data_feature.shape

(4209, 364)

In [56]:
pca.fit(test_data_feature)

PCA(n_components=0.95)

In [58]:
test_data_feature=pca.fit_transform(test_data_feature)
print(test_data_feature.shape)

(4209, 6)


In [60]:
test_pred=xgb_reg.predict(test_data_feature)
test_pred

array([ 81.16529 ,  93.338524,  86.41394 , ...,  99.643036, 107.512505,
        91.208084], dtype=float32)

In [62]:
result_2d=test_pred.reshape(-1,1)

In [64]:
result_2d.tofile('result1.csv',sep=",")

In [66]:
result_df=pd.DataFrame({'Predictions':test_pred})
result_df

,Predictions
0,81.1653
1,93.3385
2,86.4139
3,82.1776
4,104.2375
...,...
4204,118.0143
4205,93.8573
4206,99.6430
4207,107.5125


In [68]:
result_df.to_csv('tabular_output.csv')